# 99_he_week_5

Installation guide on linux: https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/week_5_batch_processing/setup/linux.md

* Recommended to be done on a remote VM

```bash
export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH"
export PYTHONPATH="${SPARK_HOME}/python/:$PYTHONPATH"
```

In [1]:
print("hello")

hello


In [2]:
import pyspark
from pyspark.sql import SparkSession

Q1: Install Spark and PySpark

In [3]:
pyspark.__version__

'3.3.2'

In [4]:
# !mkdir data

In [32]:
!rm -f data/fhvhv_tripdata_2021-06.csv.gz 
!axel https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz -o data/fhvhv_tripdata_2021-06.csv.gz

Initializing download: https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz
File size: 167.655 Megabyte(s) (175799316 bytes)
Opening output file data/fhvhv_tripdata_2021-06.csv.gz
Starting download

Connection 0 finished            2           3            ] [  13.9MB/s] [00:08]
Connection 1 finished............2     0     3            ] [  11.7MB/s] [00:07]
Connection 2 finished..................2     3     1      ] [  10.6MB/s] [00:05]
Connection 0 finished........................3     1  2   ] [  10.2MB/s] [00:03] [  10.4MB/s] [00:04]
Connection 3 finished...........................3  1  2   ] [   9.9MB/s] [00:02]
Connection 0 finished..............................1  2 3 ] [   9.8MB/s] [00:01]
Connection 1 finished................................12 3 ] [   9.7MB/s] [00:00]
Connection 0 finished..................................23 ] [   9.7MB/s] [00:00]
Connection 2 finished..................................230] [   9.7MB/s] [00:00]
Conn

In [6]:
import pandas as pd 

In [4]:
df_pd = pd.read_csv('data/fhvhv_tripdata_2021-06.csv.gz', nrows=10000)

In [14]:
df_pd.head()


,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,True,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,True,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,True,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,True,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,True,nan


In [6]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   dispatching_base_num    10000 non-null  object
 1   pickup_datetime         10000 non-null  object
 2   dropoff_datetime        10000 non-null  object
 3   PULocationID            10000 non-null  int64 
 4   DOLocationID            10000 non-null  int64 
 5   SR_Flag                 10000 non-null  object
 6   Affiliated_base_number  7286 non-null   object
dtypes: int64(2), object(5)
memory usage: 547.0+ KB


In [9]:
df_pd['Affiliated_base_number'] = df_pd['Affiliated_base_number'].astype(str)

In [13]:
df_pd['SR_Flag'] = df_pd['SR_Flag'].astype(bool)

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 14:32:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
# spark.createDataFrame(df_pd).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', BooleanType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.LongType(), True), 
    types.StructField('DOLocationID', types.LongType(), True), 
    types.StructField('SR_Flag', types.BooleanType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

  


In [33]:
# Applying custom schema to data frame
df = spark.read.format( "csv") \
    .schema(schema) \
    .option("header", True) \
    .load("data/fhvhv_tripdata_2021-06.csv.gz") 

In [34]:
df2 = df.repartition(12)

In [35]:
df2.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02395|2021-06-28 01:20:04|2021-06-28 01:33:26|          78|          69|   null|                B02395|
|              B02510|2021-06-27 20:03:51|2021-06-27 20:14:55|         186|         107|   null|                  null|
|              B02880|2021-06-21 08:35:42|2021-06-21 08:58:21|          36|         226|   null|                B02880|
|              B02510|2021-06-29 18:02:10|2021-06-29 18:34:13|         193|         239|   null|                  null|
|              B02682|2021-06-18 09:57:16|2021-06-18 10:30:37|          62|         211|   null|                B02682|
|              B02510|2021-06-25 21:25:1

In [39]:
import datetime
df.filter(
    (df.pickup_datetime > datetime.datetime(2021, 6, 15)) &
    (df.pickup_datetime < datetime.datetime(2021, 6, 16))
    ).count()

452468

In [36]:
# Building a user-defined function to get what is needed 
import datetime

def udf_calculate_diff_in_hours(dropoff_datetime, pickup_datetime):
    hours = dropoff_datetime - pickup_datetime
    return (hours.total_seconds())/(60*60)
    

In [40]:
from pyspark.sql.functions import udf
udfValueToCategory = udf(udf_calculate_diff_in_hours, types.FloatType())
df_with_cat = df2.withColumn("time_diff", udfValueToCategory(df2.dropoff_datetime, df2.pickup_datetime))

In [41]:
from pyspark.sql.functions import col
df_with_cat \
    .select( \
        col("pickup_datetime"), \
        col("dropoff_datetime"), \
        col("time_diff"))  \
    .orderBy("time_diff", ascending=False)\
    .show()
#df_with_cat.show()

+-------------------+-------------------+---------+
|    pickup_datetime|   dropoff_datetime|time_diff|
+-------------------+-------------------+---------+
|2021-06-25 13:55:41|2021-06-28 08:48:25| 66.87889|
|2021-06-22 12:09:45|2021-06-23 13:42:44|25.549723|
|2021-06-27 10:32:29|2021-06-28 06:31:20|19.980833|
|2021-06-26 22:37:11|2021-06-27 16:49:01|18.197222|
|2021-06-23 20:40:43|2021-06-24 13:08:44|16.466944|
|2021-06-23 22:03:31|2021-06-24 12:19:39|14.268888|
|2021-06-24 23:11:00|2021-06-25 13:05:35|13.909722|
|2021-06-04 20:56:02|2021-06-05 08:36:14|    11.67|
|2021-06-27 07:45:19|2021-06-27 19:07:16|11.365833|
|2021-06-20 17:05:12|2021-06-21 04:04:16|10.984445|
|2021-06-01 12:25:29|2021-06-01 22:41:32|  10.2675|
|2021-06-28 13:13:59|2021-06-28 23:11:58| 9.966389|
|2021-06-01 12:01:46|2021-06-01 21:59:45| 9.966389|
|2021-06-27 03:52:14|2021-06-27 13:30:30| 9.637777|
|2021-06-18 08:50:29|2021-06-18 18:27:57| 9.624444|
|2021-06-08 16:38:14|2021-06-09 02:07:03| 9.480278|
|2021-06-11 

In [47]:
df2.filter(
    (df.pickup_datetime >= datetime.datetime(2021, 6, 1)) &
    (df.pickup_datetime < datetime.datetime(2021, 7, 1))
    ).groupBy('PULocationID').count().orderBy('count', ascending=False).show()

+------------+------+
|PULocationID| count|
+------------+------+
|          61|231279|
|          79|221244|
|         132|188867|
|          37|187929|
|          76|186780|
|         231|164344|
|         138|161596|
|         234|158937|
|         249|154698|
|           7|152493|
|         148|151020|
|          68|147673|
|          42|146402|
|         255|143683|
|         181|143594|
|         225|141427|
|          48|139611|
|         246|139431|
|          17|138428|
|         170|137879|
+------------+------+
only showing top 20 rows



In [13]:
df2.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- SR_Flag: boolean (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [27]:
time1 = datetime.datetime(year=2021, month=6, day=1)
time2 = datetime.datetime(year=2021, month=6, day=2)

In [39]:
type(udf_calculate_diff_in_hours(time2,time1))

float

In [29]:
df_with_cat.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- SR_Flag: boolean (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)
 |-- time_diff: long (nullable = true)

